# Amazon SageMaker PyTorch コンテナを用いた MNIST の学習と推論 

(このノートブックは PyTorch の kernel で実行します）

## 目次

1. [概要](#1.概要)
1. [セットアップ](#2.セットアップ)
1. [データ](#3.データ)
1. [学習](#4.学習)
1. [SageMaker Experiments - 実験管理](#exp)
1. [ハイパーパラメータ調整を用いた学習](#hpo)
1. [SageMaker Autopilot - AutoML の実行](#auto)

---

## 1.概要

MNISTは、手書き文字の分類に広く使用されているデータセットです。 70,000個のラベル付きの28x28ピクセルの手書き数字のグレースケール画像で構成されています。 データセットは、60,000個のトレーニング画像と10,000個のテスト画像に分割されます。 手書きの数字 0から9の合計10のクラスがあります。 このチュートリアルでは、PyTorch を使用して SageMaker で MNIST モデルをトレーニングおよびテストする方法を示します。 

---

## 2.セットアップ

SageMaker セッションを作成し、設定を開始します。

- モデル学習、デプロイなどに必要なデータを格納する S3 バケットは、ノートブックインスタンス、トレーニングおよびモデルデプロイのためのインスタンスと同じリージョン内にある必要があります。
- モデル学習およびデプロイを行うために、S3にアクセスするなどの必要な権限をもった IAM ロール を指定します。 以下では、ノートブックインスタンスのロールを`sagemaker.get_execution_role()`で取得しており、これを後に利用します。別のロールを指定したい場合は `sagemaker.get_execution_role()` を適切な IAM ロール arn 文字列に置き換えてください。


In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [ ]:
import os, json
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get("DomainId")
        on_studio = True if domain_id is not None else False
print("Is this notebook runnning on Studio?: {}".format(on_studio))

## 3.データ
### 3.1.データの取得

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
if on_studio:
    !tar -xzf mnist_png.tgz -C /opt/ml --no-same-owner
else:
    !tar -xvzf  mnist_png.tgz

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

root_dir_studio = '/opt/ml'
data_dir = os.path.join(root_dir_studio,'data') if on_studio else 'data'
training_dir = os.path.join(root_dir_studio,'mnist_png/training') if on_studio else 'mnist_png/training'
test_dir = os.path.join(root_dir_studio,'mnist_png/testing') if on_studio else 'mnist_png/testing'

os.makedirs(data_dir, exist_ok=True)

training_data = datasets.ImageFolder(root=training_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

training_data_loader = DataLoader(training_data, batch_size=len(training_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

### 3.2.データをS3にアップロードする
データセットを S3 にアップロードするには、 `sagemaker.Session.upload_data` 関数を使用します。 戻り値として入力した S3 のロケーションは、後で学習ジョブを実行するときに使用します。

In [ ]:
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## 4.学習
### 4.1.学習スクリプト
`mnist.py` スクリプトは、SageMaker モデルの学習とホスティングに必要なすべてのコードを提供します（`model_fn` はモデルをロードするための関数です）。
学習スクリプトは、SageMaker の外部で実行するトレーニングスクリプトに非常に似ていますが、次のようなさまざまな環境変数を通じてトレーニング環境に関する有用なプロパティにアクセスできます。

* `SM_MODEL_DIR`： モデルアーティファクトを書き込むディレクトリへのパスを表す文字列。モデルは、推論用ホスティングのためにS3にアップロードされます。
* `SM_NUM_GPUS`： 現在のコンテナで利用可能な GPU の数。
* `SM_CURRENT_HOST`： コンテナネットワーク上の現在のコンテナの名前。
* `SM_HOSTS`： すべてのホストを含む JSON エンコードリスト。

`fit（）`メソッドの呼び出しで1つの入力チャンネル `training` が使用されたとすると、 `SM_CHANNEL_ [channel_name]`の形式に従って以下が設定されます：

* `SM_CHANNEL_TRAINING`：`training` チャンネルのデータを含むディレクトリへのパスを表す文字列。

学習に関する環境変数の詳細については、[SageMaker Containers](https://github.com/aws/sagemaker-containers) をご覧ください。

典型的なトレーニングスクリプトの書き方は下記の通りです。入力チャンネルからデータをロードし、ハイパーパラメーターで学習の設定、モデルを学習、モデルを `model_dir` に保存し、そこから学習済みモデルのホスティングを行います。 ハイパーパラメーターは引数としてスクリプトに渡され、 `argparse.ArgumentParser` インスタンスとして取得できます。

下記は、このノートブックで使われるスクリプト ` mnist.py` です。SageMaker PyTorch Container で定義されるデフォルトの実装方法に従い [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) 、 `input_fn` 、` predict_fn` 、 `output_fn` および ` transform_fn` の実装を使用します。メインガード  (``if __name__=='__main__':``)  の中のスクリプトは、学習時にのみ実行され、モデルのホスティング時には実行されません。上記のように、必要な `mnist.py` スクリプトに ` model_fn` を実装しています。

### 4.2.Estimatorの作成
学習の条件を設定するため、Estimator クラスの子クラスの PyTorch オブジェクトを作成します。 ここでは、PyTorchスクリプト、IAMロール、および（ジョブごとの）ハードウェア構成を渡す PyTorch Estimator を定義しています。また合わせてローカルの `source_dir` を指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。

`mnist.py` で定義されているハイパーパラメータをレンジの形で指定してハイパーパラメータ探索を行う場合は、[ハイパーパラメーター調整ジョブをセットアップする](### ハイパーパラメーター調整ジョブをセットアップする) で紹介しています。

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="mnist_train.py",
                    role=role,
                    source_dir = "src",
                    framework_version='1.8.1',
                    py_version='py3',
                    instance_count=1,
                    instance_type='local',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })

### 4.3.ローカルモードでの学習の実行
`fit()` メソッドで学習ジョブを実行します。`entry_point` で指定したローカルのスクリプトが、学習用のコンテナ内で実行されます。

ノートブックインスタンスのCPUで学習する場合はinstance_type = 'local'、GPUで学習する場合はlocal_gpuを指定します。インスタンス数は、ノートブックインスタンスの数、すなわち1になるため、 train_instance_countに指定された値が1より大きい場合も1として扱われますが、traini_instance_count > 1 の分散学習はローカル環境でも擬似的に検証できます。

In [ ]:
estimator.fit({'training': inputs})

### 4.4.ローカルモードでモデルの検証

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
pytorch_model = PyTorchModel(model_data = estimator.model_data,
                             entry_point='mnist_deploy.py',
                             source_dir = 'src',
                             framework_version = '1.8.1',
                             py_version = 'py3',
                             role = role)
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='local')

In [ ]:
# Select 10 images and labels from test set
import numpy as np
n_sample = 10
n_test = len(test_data)
sampled_index = np.random.choice(n_test, size=n_sample, replace=False)
sampled_testimg = np.array([test_data[i][0].numpy() for i in sampled_index])
sampled_testlabel = np.array([test_data[i][1] for i in sampled_index])

# Run inference and pick up the most likely label based on the score
prediction_scores = predictor.predict(sampled_testimg)
labels = np.argmax(prediction_scores,axis = 1)
print("Predicted labels: {}".format(labels))
print("Ground Truth:     {}".format(sampled_testlabel))

In [ ]:
predictor.delete_endpoint()

## 5.  SageMaker Experiments - 実験管理 <a id="exp"></a>

### 5.1 実験管理の全体像

実験管理は以下のコンポーネントで構成されます。
- Experiment: 複数の Trial を束ねたものです。
- Trial: 1回の実験を表し、実験設定（入力データ、パラメータなど）および実験結果（出力モデル、精度など）の1セットです。
- Trial Component: 1回の実験を任意のステップで分割して、PreprocessingやTrainingなどの名前をつけて記録できます。
- Tracker: Component の中で更に細かく情報を収集するもので、パラメータ、入出力データなどをトラックすることができます。

<img src="./figures/experiments-overview.jpg" alt="Drawing" style="width: 700px;"/>


以下では1つのExperimentの中で、2つの異なる学習率で Trial を行います。Trial Component は前処理と学習の2つに分割します。前処理に関しては、同じ入力データに対して、同じ PyTorch による標準化を行うので、異なる Trial でも同じ前処理 (同じ Trial Component)を使うことになります。学習に関しては、学習率が異なるので、異なる処理（異なる Trial Component) になります。

実験で作成されるテーブルは概ね以下のようになります。列名や数値は一例です。各行は Trial Component に対応し、それぞれ対応するComponent Name が表示されます。ここでは2種類の学習 (t1, t2) と、それぞれの前処理 (t3) が記録されています。それぞれの Trial Component が紐づく Trial も表示されます。t1とt2は、それぞれ Trial X と Y に紐づき、共通の t3 は X と Y の両方に紐づきます。Trial Component は全ての列に対して値をもっているわけではありません。例えば、学習の Trial Component に入力データに関する値を保持させず、前処理の Trial Component に保存する場合、以下のように入力データの列に NaN が表示されます。同様に、前処理の際に精度はわからないため NaN と表示されます。

| TrialComponent | ComponentName | Trial | 学習率 | 精度 (最大値) | インスタンスタイプ | 入力データの標準化 | 入力データのS3パス|
| --- | --- | --- |--- | --- | --- | --- | --- |  
| t1 | Training | X | 0.01 | 0.83 | ml.m4.xlarge | NaN | NaN | 
| t2 | Training | Y | 0.02 | 0.85 | ml.m4.xlarge | NaN | NaN | 
| t3 | Preprocessing | [X,Y] | NaN | NaN | NaN | 0.1307 | s3://data-input/examples.pt    |


それでは実験管理を行ってみましょう。まず sagemaker-experiments のライブラリをインストールして、必要なライブラリをインポートします。

In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

### 5.2 実験の作成

今回は1つの実験を作成します。実験は実験の名前と説明文をパラメータとしてもちます。上記の図で示したように、この実験に Trial Component がぶらさがります。Trial Component をぶらさげるときは実験名で指定しますので、実験名は唯一である必要があります。

In [ ]:
import time
import boto3

sm_client = boto3.Session().client('sagemaker')

mnist_experiment = Experiment.create(
    experiment_name=f"mnist-hand-written-digits-classification-{int(time.time())}", 
    description="Classification of mnist hand-written digits", 
    sagemaker_boto_client=sm_client)
print("Experiment \"{}\" is created!".format(mnist_experiment.experiment_name))

### 5.3 前処理の Trial Component の作成

この Trial Component は必須ではありませんが、通常、どういった前処理を行ったか、どういったデータを使ったかは、再現性の観点から記録しておくことが望ましいです。記録する際は、Tracker というのを作成し、log_parameters や log_input を使用して情報を入力します。Tracker で収集した情報を Trial Component にしておき、後に Trial に紐付けます。 

In [ ]:
with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm_client) as tracker:
    tracker.log_parameters({
        "normalization_mean": 0.1307,
        "normalization_std": 0.3081,
    })
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="mnist-dataset", media_type="s3/uri", value=inputs)

preprocessing_trial_component = tracker.trial_component

### 5.4 Trial の作成と Trial Component の紐付け

今回は学習率を 0.01 と 0.02 に変えて実験を行います。Trial はそれぞれの値で実行するので計2回です。変更点は学習率のみで、その他の Trial 作成方法は同じなので、for 文で作成します。for 文内の流れは以下のとおりです。

1. Trial を作成する
2. Trial に前処理の Trial Component を紐付ける
3. Trial に学習の Trial Component を紐付ける  
    3.1 通常の学習と同様に Estimator を作成する  
    3.2 Estimator に対して fit する際に Trial に紐付ける  
    
**Estimator の情報は自動的に記録されるため、Trial Component や Tracker を明示的に用意する必要はありません**




In [ ]:
from sagemaker.pytorch import PyTorch

lr_candidates = [0.01, 0.02]
for i, lr in enumerate(lr_candidates):
    # create trial
    trial_name = f"cnn-training-job-{i}-learning-rate-{int(time.time())}"
    cnn_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=mnist_experiment.experiment_name,
        sagemaker_boto_client=sm_client,
    )
    
    # associate the proprocessing trial component with the current trial
    cnn_trial.add_trial_component(preprocessing_trial_component)
    
    # all input configurations, parameters, and metrics specified in estimator 
    # definition are automatically tracked
    estimator = PyTorch(
        py_version='py3',
        entry_point='mnist_train.py',
        source_dir='src',
        role=role,
        framework_version='1.8.1',
        instance_count=1,
        instance_type='ml.c4.xlarge',
        hyperparameters={
            'epochs': 2,
            'backend': 'gloo',
            'lr': lr
        },
        metric_definitions=[
            {'Name':'test:loss', 'Regex':'Average loss: (.*?),'},
            {'Name':'test:accuracy', 'Regex':'Accuracy: [0-9]+/[0-9]+ \((.*?)%\)'}
        ],
        enable_sagemaker_metrics=True
    )
    
    cnn_training_job_name = "cnn-training-job-{}".format(int(time.time()))
    
    # Now associate the estimator with the Experiment and Trial
    estimator.fit(
        inputs={'training': inputs}, 
        job_name=cnn_training_job_name,
        experiment_config={
            "ExperimentName": mnist_experiment.experiment_name,
            "TrialName": cnn_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        wait=False,
    )
    
    # give it a while before dispatching the next training job
    time.sleep(2)

In [ ]:
cnn_trial.trial_name

### 5.5 実験結果の確認

#### 5.5.1 全体の確認

上記では学習ジョブをバックグラウンドで実行しており、すぐにこのセルに移動してきた場合、まだ学習は終わっていない可能性があります。そのような状況でも、すでに記録済みの情報、例えば、設定した学習率や入力データの情報をみることができます。

実験結果は情報が追加されるたびに更新されます。学習の実行が完了していれば、各ジョブの精度も確認できるようになります。

In [ ]:
import pandas as pd
# show 30 columns at most
pd.set_option('display.max_columns', 50)

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, 
    experiment_name=mnist_experiment.experiment_name)
df_trial = trial_component_analytics.dataframe()
df_trial

#### 5.5.2 実験結果のフィルタリング

例えば、Trial Component の Display Name が Training、つまり学習の Trial Component のみ知りたい場合は以下のようなフィルタを作成して表示します。それ以外にも、精度などの metrics やパラメータなど、表示列を絞ったり、精度の高い順にソートしたりすることができます。 

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, 
    experiment_name=mnist_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.test:accuracy.max",
    sort_order="Descending",
    metric_names=['test:accuracy','test:loss'],
    parameter_names=['lr', 'epochs', 'dropout', 'optimizer']
)
df_trial = trial_component_analytics.dataframe()
df_trial

#### 5.5.4 Lineage の確認

5.5.3 で学習の Trial の一覧を見ましたが、そのうちの1つが気になった場合、その学習に関する情報だけではなく、前処理の情報もみたくなると思います。学習も前処理も1つの Trial に結びつけていますので、Trial で検索すれば、双方を確認することができます。

上の一覧の1行目の Trial (`df_trial.Trials[0][0]`) を見たい場合は以下のようにします。Training と Preprocessing が表示されているでしょうか。

In [ ]:
focus_trial = df_trial.Trials[0][0]

lineage_table = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Equals",
            "Value": focus_trial
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_table.dataframe()

### 5.6 実験結果にもとづいてモデルをデプロイ

さきほど、実験結果に対してテストデータに対する精度の降順で、 Training の Trial Component をソートし、 `df_trial` に保存しました。つまり、`df_trial`の1行目のデータは精度が最も高い Training の情報を記録しています。最も精度の良いモデルをデプロイするため、df_trial の1行目の情報を `load` して、保存されているモデルのS3パスを `SageMaker.ModelArtifact` から取得しましょう。

In [ ]:
#Pulling best based on sort in the analytics/dataframe so first is best....

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, 
    experiment_name=mnist_experiment.experiment_name)
df_trial = trial_component_analytics.dataframe()

best_trial_component_name = df_trial.iloc[0]['TrialComponentName']
best_trial_component = TrialComponent.load(best_trial_component_name)
model_data = best_trial_component.output_artifacts['SageMaker.ModelArtifact'].value
model_data

モデルの S3 パスが分かればデプロイすることが可能です。

In [ ]:
from sagemaker.pytorch import PyTorchModel
pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='mnist_deploy.py',
                             source_dir = 'src',
                             framework_version = '1.8.1',
                             py_version = 'py3',
                             role = role,
                             name=best_trial_component.trial_component_name)
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

推論を実行してみます。

In [ ]:
# Select 10 images and labels from test set
import numpy as np
n_sample = 10
n_test = len(test_data)
sampled_index = np.random.choice(n_test, size=n_sample, replace=False)
sampled_testimg = np.array([test_data[i][0].numpy() for i in sampled_index])
sampled_testlabel = np.array([test_data[i][1] for i in sampled_index])

# Run inference and pick up the most likely label based on the score
prediction_scores = predictor.predict(sampled_testimg)
labels = np.argmax(prediction_scores,axis = 1)
print("Predicted labels: {}".format(labels))
print("Ground Truth:     {}".format(sampled_testlabel))

不要になったエンドポイントは削除しておきましょう。

In [ ]:
predictor.delete_endpoint()

## 6 ハイパーパラメータ調整を用いた学習<a id="hpo"></a>
### 6.1 ハイパーパラメーター調整ジョブの Estimator を設定
*以下のデフォルト設定では、ハイパーパラメーター調整ジョブが完了するまでに10分程度かかります。*

この例では、 SageMaker Python SDK を使用して、ハイパーパラメーターの最適化を取り入れた学習を行います。先ほどはノートブックインスタンス上で学習を行うのローカルモードを扱ったり、様々な設定を Experiments に記録しながら実行したりしました。最終的には実験結果を確認して、最も良いモデルをデプロイしました。ハイパーパラメータ調整は、実験管理で行ったようにデータやパラメータをユーザが決めて実行するのではなく、特定のデータに対して最適なパラメータを探索します。パラメータを決めるのがユーザか、SageMaker による自動化か、という点が異なります。ハイパーパラメータ調整は、とにかく手軽に最高の精度のモデルを得る目的においては便利です。完全に自動化するため、ユーザのパラメータ調整の方針を取り入れにくい点があります。

SageMaker は、学習率、バッチサイズ、エポック数などのパラメータの最適値を探索するためのインターフェースを持っています。探索したいハイパーパラメーターごとに、連続値として探索する場合はその範囲、またはリストの中から探索したい場合は探索可能な値のリストを指定します。ハイパーパラメーター調整ジョブは、異なるハイパーパラメーター設定で複数のトレーニングジョブを並列に起動し、定義済みの `objective metric` （目的関数） に基づいてそれらのトレーニングジョブの結果を評価します。以前の結果に基づいて次のハイパーパラメーター探索の設定を選択します。ハイパーパラメーター調整ジョブごとに、一度に並列で実行する学習インスタンス数、最大の学習ジョブ数、をそれぞれ割り当て、最大の学習ジョブ数が実行されると探索を終え、`objective metric` に対して最適なパフォーマンスを達成したハイパーパラメータの組み合わせを返します。

次に、以下の手順に従って、SageMaker Python SDK を使用してハイパーパラメーター調整ジョブをセットアップします。
* `estimator` を作成して PyTorch 学習ジョブをセットアップします。
* 調整するハイパーパラメーターの範囲を定義します。この例では、学習率とバッチサイズのハイパーパラメータ探索範囲を設定します。
* 最適化するチューニングジョブの `objective metric` を定義します。
* 上記の設定で `HyperparameterTuner` オブジェクトを設定します。

Spot Instanceを用いて実行する場合は、下記のコードを Estimator の train_instance_type の次の行に追加しましょう。

```python
                             max_run = 5000,
                             use_spot_instances = 'True',
                             max_wait = 10000,
```

In [ ]:
from sagemaker.pytorch import PyTorch
hpo_estimator = PyTorch(entry_point="mnist_train.py",
                    role=role,
                    source_dir ='src',
                    framework_version='1.8.1',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 4,
                        'backend': 'gloo'
                    })

`Estimator` を定義したら、調整するハイパーパラメーターとその探索範囲を指定します。ハイパーパラメーターの探索範囲の指定は3種類の方法があります。
- カテゴリーパラメーターは、探索したい値のリストを `CategoricalParameter（list）` で定義します。このリストの中から最適な値を探索します。 
- 連続パラメーターは、`ContinuousParameter（min、max）` で定義される最小値と最大値の間の連続空間で、任意の実数値から探索を行います。
- 整数パラメーターは、`IntegerParameter（min、max）` で定義された最小値と最大値の間の任意の整数値で、探索を行います。

* 可能であれば、値を最も制限の少ないタイプとして指定することをお勧めします。 たとえば、0.01〜0.2の連続値として学習率を調整すると、0.01、0.1、0.15、または0.2の値を持つカテゴリパラメーターとして調整するよりも良い結果が得られる可能性があります。 バッチサイズは一般的に2のべき乗であることが推奨されているため、ここではカテゴリパラメータとしてバッチサイズを指定しています。

In [ ]:
from sagemaker.tuner import ContinuousParameter, CategoricalParameter
hyperparameter_ranges = {'lr': ContinuousParameter(0.001, 0.1),
                         'batch-size': CategoricalParameter([32,64,128,256])}

次に objective metric とその定義を指定します。これには、トレーニングジョブの CloudWatch ログからそのメトリックを抽出するために必要な正規表現（Regex）が含まれます。 この特定のケースでは、スクリプトは平均損失値を出力し、それを objective metric として使用します。また、`objective_type` を `minimize` に設定します。これにより、ハイパーパラメーターチューニングは、最適なハイパーパラメーター設定を検索するときに客観的なメトリックを最小化しようとします。 デフォルトでは、 `objective_type` は `maximize` に設定されています。

In [ ]:
objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Average loss: (.*?),'}]

### 6.2 ハイパーパラメーター調整ジョブの tuner 設定する
次に、 `HyperparameterTuner` オブジェクトを作成します。
- 上記で作成した PyTorch 推定器
- ハイパーパラメーターの範囲
- Objective metric 名と定義
- 合計で実行するトレーニングジョブの数や並行して実行できるトレーニングジョブの数などのリソース構成の調整。

In [ ]:
from sagemaker.tuner import HyperparameterTuner
tuner = HyperparameterTuner(hpo_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

### 6.3 ハイパーパラメーター調整ジョブを起動する
`.fit（）`を呼び出し、学習およびテストデータセットへの S3 パスを渡すことで、ハイパーパラメーターチューニングジョブを開始できます。

ハイパーパラメーターチューニングジョブが作成されたら、次のステップでチューニングジョブを記述してその進行状況を確認できます。SageMaker コンソールからジョブに移動して、ハイパーパラメーターチューニングジョブの進行状況を確認できます。

In [ ]:
tuner.fit({'training': inputs})
tuner.wait()

### 6.4 ハイパーパラメータ調整の結果を確認する


In [ ]:
tuner.analytics().dataframe()

### 6.5 最良モデルをデプロイしてエンドポイントを作成する
トレーニング完了後、Tuner オブジェクトを使用して、`PyTorchPredictor` をビルドおよびデプロイします。前の手順では、Tuner が複数のトレーニングジョブを起動し、最適な obejctive metric を持つ結果のモデルが最適なモデルとして定義しました。これにより、SageMaker エンドポイントが作成されます。これにより、チューナーが探索した最適なモデルに基づいて推論を実行するためのホスティングを行います。

tuner.best_estimator() によって、最善の Estimator の情報を取り出すことができ、そこから model_data で最良のモデルへのパスを調べます。
それ以降はこれまでの deploy と同じです。

In [ ]:
from sagemaker.pytorch import PyTorchModel
model_data = tuner.best_estimator().model_data
pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='mnist_deploy.py',
                             source_dir = 'src',
                             framework_version = '1.8.1',
                             py_version = 'py3',
                             role = role,
                             name=best_trial_component.trial_component_name)
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

### 6.6 評価

In [ ]:
# Select 10 images and labels from test set
import numpy as np
n_sample = 10
n_test = len(test_data)
sampled_index = np.random.choice(n_test, size=n_sample, replace=False)
sampled_testimg = np.array([test_data[i][0].numpy() for i in sampled_index])
sampled_testlabel = np.array([test_data[i][1] for i in sampled_index])

# Run inference and pick up the most likely label based on the score
prediction_scores = predictor.predict(sampled_testimg)
labels = np.argmax(prediction_scores,axis = 1)
print("Predicted labels: {}".format(labels))
print("Ground Truth:     {}".format(sampled_testlabel))

In [ ]:
predictor.delete_endpoint()

## 7. SageMaker Autopilot - AutoML の実行<a id="auto"></a>

(注意：AutoMLは30分程度かかります）

### 7.1 CSV ファイルの用意

Autopilot は 表データ (CSV ファイル) に対して実行することができます。そこでまず mnist のデータである 28x28 の画像を 784 個の1行のデータにして、データ数 60000 x 画像 784 の表データに変換します。加えて、785列目にラベルの情報を加えます。

ヘッダを $x_0,x_1, \cdots, x_{783}, y$ として与えて、csv 形式で保存します。

In [ ]:
from torch.utils.data import DataLoader
import pandas as pd

# Create 60000x 785 matrix. 785 = 784 features + 1 label. 
all_train_loader = DataLoader(training_data, batch_size=len(training_data))
X_train = next(iter(all_train_loader))[0].numpy()
y_train = next(iter(all_train_loader))[1].numpy()
X_train = X_train.reshape([60000, 28*28])
y_train = y_train.reshape([60000,1])
train_csv = np.concatenate([X_train,y_train], axis = 1)

# Headers are x0, x1, ... x,783, y
headers = ["x" + str(i) for i in range(28*28)] + ["y"]

# Create dataframe with headers to write csv
train_df = pd.DataFrame(train_csv, columns = headers)
train_df['y'] = train_df['y'].astype(int)
train_df.to_csv("mnist.csv", index=False)
train_df.head()

### 7.2 ファイルのアップロード

In [ ]:
input_csv = sagemaker_session.upload_data(path='mnist.csv', bucket=bucket, key_prefix=prefix)

### 7.3 AutoML の実行

CSV の中でラベルの列名 `y` や、最大実行回数 `max_candidates` を指定して、先程アップロードした csv に対して fit すれば AutoML を実行可能です。

In [ ]:
from sagemaker import AutoML
from time import gmtime, strftime, sleep

timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
base_job_name = 'automl-mnist-sdk-' + timestamp_suffix

target_attribute_name = 'y'

automl = AutoML(role=role,
                target_attribute_name=target_attribute_name,
                base_job_name=base_job_name,
                sagemaker_session=sagemaker_session,
                max_candidates=2)
automl.fit(input_csv, job_name=base_job_name, wait=False, logs=False)

### 7.4 実行状況の確認

describe_auto_ml_job()を利用すると、AutoML の実行状況を見ることが可能です。以下では、AutoML の実行状況を InProgress のあいだ、監視し続けます。

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = automl.describe_auto_ml_job()
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = automl.describe_auto_ml_job()
    job_run_status = describe_response['AutoMLJobStatus']
    
    print(describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

### 7.5 最良の学習モデルの確認

describe_auto_ml_job() の `BestCandidate` から最良のモデルに関する情報を取得することができます。

In [ ]:
best_candidate = automl.describe_auto_ml_job()['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

### 7.6 最良のモデルでバッチ推論を行う

テストデータの CSV を用意して、それらに対するラベルを最良のモデルで一括して推論しましょう。まずはテストデータの CSV を用意します。テストデータは header が不要です。

In [ ]:
# Create 10000 x 784 matrix. 
all_test_loader = DataLoader(test_data, batch_size=len(test_data))
X_test = next(iter(all_test_loader))[0].numpy()
X_test = X_test.reshape([10000, 28*28])
np.savetxt("mnist_test.csv", X_test, delimiter=",")
input_test_csv = sagemaker_session.upload_data(path='mnist_test.csv', bucket=bucket, key_prefix=prefix)

best_candudidate から最良のモデルデータを取り出し、model に保存します。バッチ推論を行うためには、model に対して transformer を作成して、transform を実行します。1行ごとにデータが記載されているので、split_type を Line に、 content_type は text/csv にします。

In [ ]:
s3_transform_output_path = 's3://{}/{}/inference-results/'.format(bucket, prefix);
inference_response_keys = ['predicted_label', 'probability']

model = automl.create_model(name=best_candidate['CandidateName'],
                            candidate=best_candidate,
                            inference_response_keys=inference_response_keys)
    
output_path = s3_transform_output_path + best_candidate['CandidateName'] +'/'
    
transformer = model.transformer(instance_count=1, 
                  instance_type='ml.m5.xlarge',
                  assemble_with='Line',
                  output_path=output_path)

transformer.transform(data=input_test_csv, split_type='Line', content_type='text/csv', wait=False)
print("Starting transform job {}".format(transformer._current_job_name))


## 7.7 評価

しばらくするとバッチ推論を行った結果が S3 に保存されますので、それを get_csv_from_s3 でダウンロードします。0列目は推定ラベル、1列目はその確からしさです。

In [ ]:
import json
import io
from urllib.parse import urlparse

def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip('/')
    s3 = boto3.resource('s3')
    obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))
    return obj.get()["Body"].read().decode('utf-8')

pred_csv = get_csv_from_s3(transformer.output_path, '{}.out'.format("mnist_test.csv"))
prediction = pd.read_csv(io.StringIO(pred_csv), header=None)
prediction

推論結果と正解を比較しましょう。まず正解データを取り出します。

In [ ]:
y_test = next(iter(all_test_loader))[1].numpy()

scikit-learnを使って評価します。

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction.values[:,0])

In [ ]:
best_candidate['CandidateName']